# 處理 outliers
* 新增欄位註記
* outliers 或 NA 填補
    1. 平均數 (mean)
    2. 中位數 (median, or Q50)
    3. 最大/最小值 (max/min, Q100, Q0)
    4. 分位數 (quantile)

# [作業目標]
- 仿造範例的資料操作, 試著進行指定的離群值處理

# [作業重點]
- 計算 AMT_ANNUITY 的分位點 (q0 - q100) (Hint : np.percentile, In[3])
- 將 AMT_ANNUITY 的 NaN 用中位數取代 (Hint : q50, In[4])
- 將 AMT_ANNUITY 數值轉換到 -1 ~ 1 之間 (Hint : 參考範例, In[5])
- 將 AMT_GOOD_PRICE 的 NaN 用眾數取代 (In[6])

In [1]:
# Import 需要的套件
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# 設定 data_path
dir_data = './Part01/'

In [2]:
f_app = os.path.join(dir_data, 'application_train.csv')
print('Path of read in data: %s' % (f_app))
app_train = pd.read_csv(f_app)
app_train.head()

Path of read in data: ./Part01/application_train.csv


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


## 1. 列出 AMT_ANNUITY 的 q0 - q100
## 2.1 將 AMT_ANNUITY 中的 NAs 暫時以中位數填補
## 2.2 將 AMT_ANNUITY 的數值標準化至 -1 ~ 1 間
## 3. 將 AMT_GOOD_PRICE 的 NAs 以眾數填補


In [5]:
# 如果欄位中有 NA, describe 會有問題
app_train['AMT_ANNUITY'].describe()

# Ignore NA
five_num = [0,  100]
quantile = [np.percentile(app_train[~app_train['AMT_ANNUITY'].isnull()]['AMT_ANNUITY'], q = i) for i in five_num]
print(quantile)

# 1: 計算 AMT_ANNUITY 的 q0 - q100
q_all = quantile[0] - quantile[1]

pd.DataFrame({'q': list(range(101)),
              'value': q_all})

[1615.5, 258025.5]


,q,value
0,0,-256410.0
1,1,-256410.0
2,2,-256410.0
3,3,-256410.0
4,4,-256410.0
5,5,-256410.0
6,6,-256410.0
7,7,-256410.0
8,8,-256410.0
9,9,-256410.0


In [6]:
# 2.1 將 NAs 以 q50 填補
print("Before replace NAs, numbers of row that AMT_ANNUITY is NAs: %i" % sum(app_train['AMT_ANNUITY'].isnull()))

q_50 = [np.percentile(app_train[~app_train['AMT_ANNUITY'].isnull()]['AMT_ANNUITY'], q = 50) ]
app_train.loc[app_train['AMT_ANNUITY'].isnull(),'AMT_ANNUITY'] = q_50

print("After replace NAs, numbers of row that AMT_ANNUITY is NAs: %i" % sum(app_train['AMT_ANNUITY'].isnull()))

Before replace NAs, numbers of row that AMT_ANNUITY is NAs: 12
After replace NAs, numbers of row that AMT_ANNUITY is NAs: 0


### Hints: Normalize function (to -1 ~ 1)
$ y = 2*(\frac{x - min(x)}{max(x) - min(x)} - 0.5) $

In [12]:
# 2.2 Normalize values to -1 to 1
print("== Original data range ==")
print(app_train['AMT_ANNUITY'].describe())
value = app_train['AMT_CREDIT'].values

def normalize_value(x):
    value = x.values
    x = 2*( value - min(value) ) / ( max(value) - min(value) -0.5)
    return x

app_train['AMT_ANNUITY_NORMALIZED'] = normalize_value(app_train['AMT_ANNUITY'])

print("== Normalized data range ==")
app_train['AMT_ANNUITY_NORMALIZED'].describe()

== Original data range ==
count    307511.000000
mean          0.276667
std           0.200994
min           0.000000
25%           0.112360
50%           0.233973
75%           0.381348
max           2.000000
Name: AMT_ANNUITY, dtype: float64
== Normalized data range ==


count    307511.000000
mean          0.368890
std           0.267992
min           0.000000
25%           0.149813
50%           0.311964
75%           0.508464
max           2.666667
Name: AMT_ANNUITY_NORMALIZED, dtype: float64

In [13]:
# 3
print("Before replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: %i" % sum(app_train['AMT_GOODS_PRICE'].isnull()))

# 列出重複最多的數值
# 計算眾數 (mode)
# 較快速的方式*****
from collections import defaultdict
mode_dict = defaultdict(lambda:0)
for value in app_train[~app_train['AMT_ANNUITY'].isnull()]['AMT_ANNUITY']:
    mode_dict[value] += 1
value_most = sorted(mode_dict.items(), key=lambda kv: kv[1], reverse=True)
print(value_most)

mode_goods_price = list(app_train['AMT_GOODS_PRICE'].value_counts().index)
app_train.loc[app_train['AMT_GOODS_PRICE'].isnull(), 'AMT_GOODS_PRICE'] = mode_goods_price[0]

print("After replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: %i" % sum(app_train['AMT_GOODS_PRICE'].isnull()))

Before replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: 278
[(0.20224721626057632, 9709), (0.3146067808497854, 8877), (0.08988765167136725, 8162), (0.06741573875352544, 7342), (0.11235956458920907, 7241), (0.4269663454389944, 6246), (0.10471911419714285, 4500), (0.24970789634305823, 4437), (0.38134836221577556, 4152), (0.044943825835683626, 3660), (0.35465172966937947, 3524), (0.5393259100282035, 3493), (0.23784272632243775, 3175), (0.11955057672291844, 3096), (0.515955120593648, 2856), (0.20449440755236048, 2649), (0.6208989539199693, 2613), (0.11213484546003065, 2463), (0.07865169521244635, 2339), (0.3680000459425775, 2328), (0.6516854746174126, 2321), (0.09730338293425504, 2084), (0.13483147750705088, 2072), (0.10112360813028816, 2057), (0.2971685764255401, 2041), (0.4803596105317866, 1883), (0.24719104209625994, 1821), (0.1573033904248927, 1743), (0.23145845586247887, 1717), (0.27343823638429915, 1688), (0.22597755630181726, 1665), (0.1797753033427345, 1661), (0.39469667848